<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/tools_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/tools_numpy.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

[Video de apoyo a la lectura de este cuaderno](https://www.youtube.com/watch?v=j6buYeCOZ14)


# Clase 2



# **Procesamiento y edición de bases de datos con Python**



## Traducción entre Python, Excel y Stata





# Análisis de datos sobre clases sociales y estratificación  


## Clase 2: Procesamiento y edición de bases de datos con Python



**Curso:** Análisis de datos sobre clases sociales y estratificación con Python, Stata e IA  


**Modalidad:** En video.   
**Fecha:**  2 de Enero de 2026  
**Docente:** Marco Julio Cañas  
**Duración:** 30 minutos  

---



### Objetivos de la clase

Al finalizar esta sesión, el estudiante será capaz de:

- Importar bases de datos en distintos formatos (CSV, Excel, Stata).
- Reconocer la estructura de una base de datos de encuesta.
- Editar, seleccionar y recodificar variables.
- Exportar datos para su uso en Stata.
- Comprender el flujo de trabajo Python ↔ Stata.




# 1. ¿Por qué procesar datos antes de analizarlos?



En la investigación social cuantitativa:

- Los datos **nunca** están listos para analizar.
- El procesamiento es una etapa central del trabajo empírico.
- Decisiones técnicas tienen consecuencias teóricas.



En esta clase trabajaremos:
- Limpieza
- Selección
- Recodificación
- Traducción entre softwares

  - Python será usado como **lenguaje de preparación de datos**  
  - Stata será usado luego como **lenguaje de análisis estadístico**



# 2. Importación de librerías (Código)


In [24]:


import pandas as pd
import numpy as np




# 3. Importación de bases de datos



Python permite importar datos en múltiples formatos:

- CSV
- Excel
- Stata (.dta)

Trabajaremos con un ejemplo de encuesta simplificada.


---


# Generación del archivo de encuesta ejemplo (Código)  



Vamos a generar el archivo `encuesta_ejemplo.csv` debe ser una **encuesta social simplificada**, con las variables que luego se usan para:

* exploración (`head`, `info`)
* edición
* creación de variables (p. ej. `cohorte_edad`)
* exportación a Excel y Stata



En el notebook se trabaja explícitamente con las siguientes columnas:

* `edad`
* `genero`
* `educacion`
* `ingreso`
* `ocupado`



# Script en Python para generar `encuesta_ejemplo.csv`




In [25]:
import numpy as np
import pandas as pd


In [26]:

# -----------------------------
# 1. Fijar semilla (reproducibilidad)
# -----------------------------
np.random.seed(42)

# -----------------------------
# 2. Tamaño de la muestra
# -----------------------------
n = 120  # tamaño típico para ejercicios de clase

# -----------------------------
# 3. Generación de variables
# -----------------------------

# Edad: adultos jóvenes y medios
edad = np.random.randint(low = 18, high = 60, size = n)

# Género
genero = np.random.choice(
    ["Hombre", "Mujer"],
    size=n,
    p=[0.48, 0.52]
)

# Nivel educativo
educacion = np.random.choice(
    ["Primaria", "Secundaria", "Técnica", "Universitaria"],
    size=n,
    p=[0.20, 0.35, 0.20, 0.25]
)

# Ingreso mensual (en pesos colombianos, distribución sesgada)
ingreso = np.random.normal(
    loc=1_800_000,  # ingreso promedio
    scale = 600_000, # dispersión
    size = n
).round(0)

# Evitar ingresos negativos
ingreso = np.clip(ingreso, min = 500000, max = None)

# Ocupación (variable binaria)
ocupado = np.random.choice(
    ["Sí", "No"],
    size=n,
    p=[0.65, 0.35]
)

# -----------------------------
# 4. Construcción del DataFrame
# -----------------------------
df = pd.DataFrame({
    "edad": edad,
    "genero": genero,
    "educacion": educacion,
    "ingreso": ingreso.astype(int),
    "ocupado": ocupado
})

# -----------------------------
# 5. Introducir algunos valores faltantes (didáctico)
# -----------------------------
for col in ["ingreso", "educacion"]:
    idx = np.random.choice(df.index, size=5, replace=False)
    df.loc[idx, col] = np.nan

# -----------------------------
# 6. Guardar archivo CSV
# -----------------------------
df.to_csv("encuesta_ejemplo.csv", index=False)

print("Archivo 'encuesta_ejemplo.csv' generado correctamente.")
df.head()


Archivo 'encuesta_ejemplo.csv' generado correctamente.


,edad,genero,educacion,ingreso,ocupado
0,56,Mujer,Secundaria,1657565.0,Sí
1,46,Mujer,Técnica,2923203.0,No
2,32,Mujer,Secundaria,1788335.0,No
3,25,Mujer,Secundaria,1240445.0,Sí
4,38,Hombre,Técnica,1023035.0,Sí


# Coherencia con diseño de clase



Este dataset permite trabajar exactamente lo que propones en la **Clase 2**:

* ✔ Importación de CSV
* ✔ Exploración inicial (`head`, `info`, `describe`)
* ✔ Variables categóricas y numéricas
* ✔ Datos faltantes
* ✔ Creación de cohortes de edad con `pd.cut`
* ✔ Exportación a Excel y Stata (`.dta`) para la Clase 3



# 4. Cargar datos desde un archivo CSV (Código)


In [41]:

# Ejemplo: cargar un archivo CSV
# (en Colab, subir previamente el archivo)

df = pd.read_csv("encuesta_ejemplo.csv")
df.head()



,edad,genero,educacion,ingreso,ocupado
0,56,Mujer,Secundaria,1657565.0,Sí
1,46,Mujer,Técnica,2923203.0,No
2,32,Mujer,Secundaria,1788335.0,No
3,25,Mujer,Secundaria,1240445.0,Sí
4,38,Hombre,Técnica,1023035.0,Sí



*(Si no se tiene archivo, se puede crear uno ficticio como sigue)*


In [28]:
df = pd.DataFrame({
    "edad": [23, 45, 34, 52, 29],
    "genero": [1, 2, 1, 2, 1],
    "educacion": [2, 4, 3, 1, 4],
    "ingreso": [1200000, 3500000, 2200000, 1800000, 2600000],
    "ocupado": [1, 1, 1, 0, 1]
})

df


,edad,genero,educacion,ingreso,ocupado
0,23,1,2,1200000,1
1,45,2,4,3500000,1
2,34,1,3,2200000,1
3,52,2,1,1800000,0
4,29,1,4,2600000,1



# 5. Exploración inicial

Antes de analizar:
- ¿Cuántos casos hay?
- ¿Qué variables existen?
- ¿Qué tipos de datos tenemos?




# 6. Exploración básica (Código)


In [29]:

df.shape



(5, 5)

In [30]:

df.columns



Index(['edad', 'genero', 'educacion', 'ingreso', 'ocupado'], dtype='object')

In [31]:

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   edad       5 non-null      int64
 1   genero     5 non-null      int64
 2   educacion  5 non-null      int64
 3   ingreso    5 non-null      int64
 4   ocupado    5 non-null      int64
dtypes: int64(5)
memory usage: 332.0 bytes



# 7. Selección y renombrado de variables



En encuestas reales:
- Sobran variables
- Los nombres suelen ser poco claros

Buena práctica:
- Quedarse solo con lo necesario
- Usar nombres comprensibles



# 8. Selección de variables (Código)


In [32]:

df = df[["edad", "genero", "educacion", "ingreso", "ocupado"]]
df


,edad,genero,educacion,ingreso,ocupado
0,23,1,2,1200000,1
1,45,2,4,3500000,1
2,34,1,3,2200000,1
3,52,2,1,1800000,0
4,29,1,4,2600000,1


# 9. Recodificación de variables



Ejemplo:
- género: 1 = Mujer, 2 = Hombre
- ocupado: 1 = Sí, 0 = No



Estas decisiones son **analíticas**, no solo técnicas.




# 10. Recodificación en Python (Código)


In [36]:
df.educacion.unique()

array(['Secundaria', 'Técnica', 'Universitaria', 'Primaria', nan],
      dtype=object)

In [42]:

df["genero"] = df["genero"].map({"Mujer":1, "Hombre":2})
df["ocupado"] = df["ocupado"].map({"Sí":1, "No":0})
df['educacion'] = df['educacion'].map({'Primaria':1, 'Secundaria':2, 'Técnica':3, 'Universitaria':4})
df



,edad,genero,educacion,ingreso,ocupado
0,56,1,2.0,1657565.0,1
1,46,1,3.0,2923203.0,0
2,32,1,2.0,1788335.0,0
3,25,1,2.0,1240445.0,1
4,38,2,3.0,1023035.0,1
...,...,...,...,...,...
115,54,2,2.0,2017486.0,1
116,31,2,2.0,2142809.0,0
117,20,1,2.0,2382314.0,1
118,18,1,4.0,2245178.0,1



## 11. Creación de nuevas variables

Ejemplo:
- Cohortes de edad
- Variables clave en estudios de desigualdad



# 12. Crear cohortes de edad (Código)


In [43]:

bins = [18, 30, 45, 60]
labels = ["18-29", "30-44", "45-59"]

df["cohorte_edad"] = pd.cut(df["edad"], bins=bins, labels=labels)
df


,edad,genero,educacion,ingreso,ocupado,cohorte_edad
0,56,1,2.0,1657565.0,1,45-59
1,46,1,3.0,2923203.0,0,45-59
2,32,1,2.0,1788335.0,0,30-44
3,25,1,2.0,1240445.0,1,18-29
4,38,2,3.0,1023035.0,1,30-44
...,...,...,...,...,...,...
115,54,2,2.0,2017486.0,1,45-59
116,31,2,2.0,2142809.0,0,30-44
117,20,1,2.0,2382314.0,1,18-29
118,18,1,4.0,2245178.0,1,NaN


# ¿Por qué al crear el atributo con pd.cut() se generaron 4 datos faltantes?


Muy buena pregunta, y además **didácticamente clave**. 


La respuesta está en **cómo funciona exactamente `pd.cut()` respecto a los intervalos**.

---

## 1. Qué hace `pd.cut()` por defecto

Cuando escribes:


In [ ]:

```python
bins = [18, 30, 45, 60]
labels = ["18-29", "30-44", "45-59"]

df["cohorte_edad"] = pd.cut(df["edad"], bins=bins, labels=labels)
```



Pandas construye **intervalos semiabiertos** del tipo:

$$
(a, b]
$$

Es decir:

| Intervalo real | Etiqueta |
| -------------- | -------- |
| (18, 30]       | "18-29"  |
| (30, 45]       | "30-44"  |
| (45, 60]       | "45-59"  |

🔴 **El límite inferior NO se incluye**
🟢 **El límite superior SÍ se incluye**

---



## 2. Qué edades quedan fuera (y generan `NaN`)

En el dataset que generamos:



```python
edad = np.random.randint(18, 60, size=n)
```



⚠️ `np.random.randint(18, 60)` **sí incluye 18**, pero **no incluye 60**
Por tanto, algunas personas tienen:

```text
edad == 18
```



Ahora observa:

* 18 **NO pertenece** al intervalo `(18, 30]`
* Por tanto → `NaN`

Si en tu dataset aparecen **4 personas con edad = 18**, obtienes exactamente:

```text
4 valores faltantes en cohorte_edad
```



✔️ El comportamiento es correcto
✔️ No es un error del script
✔️ Es un excelente ejemplo para enseñanza

---



## 3. Cómo verificarlo empíricamente (muy pedagógico)


In [ ]:

```python
df[df["cohorte_edad"].isna()]["edad"].value_counts()
```



Verás algo como:

```text
18    4
```

---



# 4. Cómo corregirlo (dos formas correctas)



## Opción A — Incluir el límite inferior


In [ ]:

df["cohorte_edad"] = pd.cut(
    df["edad"],
    bins=bins,
    labels=labels,
    include_lowest=True
)




Ahora los intervalos quedan:

| Intervalo |
| --------- |
| [18, 30]  |
| (30, 45]  |
| (45, 60]  |

✔️ El 18 ya queda incluido

---



## Opción B — Redefinir los bins (más explícito para estudiantes)


In [50]:


bins = [17, 29, 44, 59]
labels = ["18-29", "30-44", "45-59"]

df["cohorte_edad"] = pd.cut(df["edad"], bins=bins, labels=labels)




Aquí haces coincidir **exactamente** los valores enteros posibles con las cohortes.

---



## 5. Lectura conceptual (muy útil para tu curso)



Este fenómeno permite enseñar que:

> **La discretización no es neutra**
> Toda categorización implica decisiones matemáticas y conceptuales.

Puedes formularlo así en clase:

* ¿Quién queda *dentro* y quién queda *fuera*?
* ¿Por qué los extremos importan?
* ¿Qué pasa si el dato es continuo vs. discreto?

---



## 6. Frase para el notebook (lista para pegar)

> *Los valores faltantes generados por `pd.cut()` no indican un error en los datos, sino una decisión implícita sobre los límites de los intervalos. Comprender esta decisión es parte esencial del análisis de datos.*

Si quieres, en el siguiente mensaje puedo ayudarte a **convertir esto en una actividad guiada** o una **pregunta de evaluación conceptual** para tus estudiantes de Licenciatura 📘📊


# 13. Datos faltantes

En encuestas:
- Los datos faltantes son frecuentes
- Deben identificarse explícitamente




# 14. Identificación de valores faltantes (Código)


In [45]:
df['edad'].describe()

count    120.000000
mean      38.050000
std       12.444735
min       18.000000
25%       26.000000
50%       39.000000
75%       50.000000
max       59.000000
Name: edad, dtype: float64

In [46]:

df.isna().sum()



edad            0
genero          0
educacion       5
ingreso         5
ocupado         0
cohorte_edad    4
dtype: int64

# 15. Exportar datos para otros softwares



Una vez procesados los datos en Python:
- Se exportan para análisis en Stata
- Se garantiza reproducibilidad




# 16. Exportar a Excel (Código)


In [48]:


df.to_excel("encuesta_procesada.xlsx", index=False)




# 17. Exportar a Stata (.dta) (Código)


In [49]:

df.to_stata("encuesta_procesada.dta", write_index=False)




- Este archivo será usado **directamente en Stata** en la Clase 3.

---



## 18. Uso de IA en esta etapa

La IA puede ayudar a:
- Explicar código
- Detectar errores
- Traducir sintaxis Python ↔ Stata

Ejemplo de consigna:
"Explícale a un estudiante de sociología qué hace este script y por qué es importante."




# 19. Actividad práctica

1. Importar una base de datos (real o ficticia).
2. Seleccionar al menos 5 variables.
3. Recodificar:
   - una variable categórica
   - una variable binaria
4. Crear una variable nueva relevante para desigualdad social.
5. Exportar la base procesada a formato Stata (.dta).




## 20. Cierre

En esta clase aprendimos:
- A procesar bases de datos con Python
- A tomar decisiones analíticas en la limpieza
- A traducir datos entre Python, Excel y Stata

Próxima clase:
* Análisis descriptivo de variables con Stata  
* Construcción de variables complejas




# Nota pedagógica final

Este cuaderno:

* Introduce el **flujo de trabajo real de investigación**
* Prepara el terreno para Stata
* Refuerza la reproducibilidad
* Se integra naturalmente con IA

---
